In [34]:
from dotenv import load_dotenv
load_dotenv() 

import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [35]:
from src.inference import get_feature_store

In [36]:
feature_store = get_feature_store()

2025-05-10 11:27:52,245 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 11:27:52,334 INFO: Initializing external client
2025-05-10 11:27:52,335 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 11:27:52,939 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1228969


In [37]:
from datetime import datetime, timedelta
import pandas as pd  


current_date = pd.Timestamp.now()
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
# Ensure fetch_data_from and fetch_data_to are timezone-aware to match ts_data.pickup_hour
fetch_data_from = fetch_data_from.tz_localize(None)
fetch_data_to = fetch_data_to.tz_localize(None)
ts_data.pickup_hour = ts_data.pickup_hour.dt.tz_localize(None)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-05-10 11:27:54,047 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 11:27:54,054 INFO: Initializing external client
2025-05-10 11:27:54,055 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 11:27:54,572 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1228969
Fetching data from 2025-04-11 11:27:54.045222 to 2025-05-10 10:27:54.045222
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.70s) 


In [ ]:
ts_data.sort_values(["station_id", "pickup_hour"]).reset_index(drop=True)

In [ ]:
ts_data

In [ ]:
from src.data_utils import transform_ts_data_info_features_and_target

features , _ = transform_ts_data_info_features_and_target(ts_data, window_size=28, step_size=23)